In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, when, coalesce, lit, 
    avg, min, max, sum as sum_col, countDistinct,
    current_timestamp
)

# 创建Spark会话
spark = SparkSession.builder \
    .appName("Gold Layer - Feature Engineering") \
    .config("spark.driver.memory", "2g") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .getOrCreate()

# 定义Silver层表路径
silver_tables = {
    "loan": "datamart/silver/silver_loan",
    "attributes": "datamart/silver/silver_attributes",
    "financials": "datamart/silver/silver_financials",
    "clickstream_detailed": "datamart/silver/silver_clickstream_detailed",
    "clickstream_aggregated": "datamart/silver/silver_clickstream_aggregated"
}

# 加载所有Silver层表
print("加载Silver层表...")
silver_dfs = {}
for table_name, path in silver_tables.items():
    try:
        df = spark.read.parquet(path)
        silver_dfs[table_name] = df
        print(f"已加载 {table_name} 表, 行数: {df.count()}")
    except Exception as e:
        print(f"读取{table_name}表时出错: {e}")

# 步骤1: 创建基础客户表，包含所有客户
print("\n步骤1: 创建基础客户表...")

# 从属性表中获取所有客户ID
base_customers = silver_dfs['attributes'].select("Customer_ID")
print(f"基础客户表中的客户数: {base_customers.count()}")

# 标记哪些客户有点击流数据
clickstream_customers_df = silver_dfs['clickstream_aggregated'].select("Customer_ID").distinct()

# 将客户表与点击流客户表左外连接，创建标志字段
base_customers_with_flag = base_customers.join(
    clickstream_customers_df,
    "Customer_ID",
    "left"
).withColumn(
    "has_clickstream_data",
    when(clickstream_customers_df.Customer_ID.isNotNull(), True).otherwise(False)
)

# 检查标志分布
print("\n点击流数据可用性分布:")
base_customers_with_flag.groupBy("has_clickstream_data").count().show()

# 步骤2: 合并客户的属性和财务数据
print("\n步骤2: 合并客户属性和财务数据...")

# 获取客户属性数据（从silver_attributes表）
attributes_df = silver_dfs['attributes'].select(
    "Customer_ID",
    col("age"),
    col("occupation"),
    col("is_valid_age")
)

# 获取客户财务数据（从silver_financials表）
# 选择最重要的财务特征
financials_df = silver_dfs['financials'].select(
    "Customer_ID",
    col("annual_income"),
    col("Monthly_Inhand_Salary"),
    col("Num_Bank_Accounts"),
    col("Num_Credit_Card"),
    col("Interest_Rate"),
    col("num_of_loans"),
    col("Credit_Utilization_Ratio"),
    col("credit_history_total_months"),
    col("outstanding_debt"),
    col("num_delayed_payments"),
    col("credit_mix"),
    col("payment_behavior_spent_level"),
    col("payment_behavior_value_level")
)

# 合并属性和财务数据
customer_profile_df = base_customers_with_flag.join(
    attributes_df, 
    "Customer_ID", 
    "left"
).join(
    financials_df, 
    "Customer_ID", 
    "left"
)

# 查看合并结果
print(f"客户属性和财务数据合并结果 - 行数: {customer_profile_df.count()}, 列数: {len(customer_profile_df.columns)}")

# 步骤3: 添加点击流数据
print("\n步骤3: 添加点击流数据...")

# 获取点击流聚合数据
# 从众多点击流特征中选择最有代表性的几个
clickstream_agg_df = silver_dfs['clickstream_aggregated'].select(
    "Customer_ID",
    # 为每个特征选择均值、标准差和异常值计数
    # 这里只使用前5个特征作为示例
    *[col(f"fe_{i}_mean") for i in range(1, 6)],
    *[col(f"fe_{i}_stddev") for i in range(1, 6)],
    *[col(f"fe_{i}_outlier_count") for i in range(1, 6)],
    col("record_count")
)

# 将点击流数据左连接到客户资料表
customer_profile_with_clicks_df = customer_profile_df.join(
    clickstream_agg_df, 
    "Customer_ID", 
    "left"
)

# 处理缺失的点击流特征
# 对于没有点击流数据的客户，我们可以使用特殊值填充
processed_df = customer_profile_with_clicks_df

# 填充策略: 使用0填充缺失的点击流特征
for feature in clickstream_agg_df.columns:
    if feature != "Customer_ID":
        processed_df = processed_df.withColumn(
            feature,
            coalesce(col(feature), lit(0))  # 如果值为NULL，使用0填充
        )

# 添加表明特征来源的元数据
processed_df = processed_df.withColumn(
    "clickstream_features_imputed", 
    when(col("has_clickstream_data") == False, True).otherwise(False)
)

print(f"添加点击流数据后 - 行数: {processed_df.count()}, 列数: {len(processed_df.columns)}")

# 步骤4: 添加贷款特征和创建标签
print("\n步骤4: 添加贷款特征和创建标签...")

# 为每个客户计算贷款汇总统计
loan_summary = silver_dfs['loan'].groupBy("Customer_ID").agg(
    # 计算贷款总数
    countDistinct("loan_id").alias("total_loans"),
    # 计算违约贷款数量
    sum_col(when(col("is_default") == True, 1).otherwise(0)).alias("num_defaulted_loans"),
    # 计算平均逾期次数
    avg("consecutive_missed_payments").alias("avg_missed_payments"),
    # 计算最大连续逾期次数
    max("consecutive_missed_payments").alias("max_missed_payments"),
    # 计算平均已付款百分比
    avg("percent_paid").alias("avg_percent_paid"),
    # 获取最大逾期金额
    max("overdue_amt").alias("max_overdue_amt"),
    # 获取最小账户余额
    min("balance").alias("min_balance")
)

# 将贷款汇总统计添加到客户资料表
gold_features_df = processed_df.join(
    loan_summary, 
    "Customer_ID", 
    "left"
)

# 创建目标变量(标签) - 是否有违约贷款
gold_features_df = gold_features_df.withColumn(
    "has_defaulted", 
    col("num_defaulted_loans") > 0
)

# 创建更全面的违约风险评分 (0-120之间的值)
gold_features_df = gold_features_df.withColumn(
    "default_risk_score",
    (
        # 基础分数 - 根据违约状态设置基础分数
        when(col("num_defaulted_loans") > 0, 50).otherwise(0) +
        
        # 违约历史因素 (0-20分)
        (col("num_defaulted_loans") * 5).cast("int") + 
        
        # 逾期行为因素 (0-15分)
        (col("max_missed_payments") * 1.5).cast("int") + 
        
        # 债务负担因素 (0-10分)
        when(
            (col("outstanding_debt") / col("annual_income") * 100) > 50, 10
        ).when(
            (col("outstanding_debt") / col("annual_income") * 100) > 30, 6
        ).when(
            (col("outstanding_debt") / col("annual_income") * 100) > 10, 3
        ).otherwise(0) +
        
        # 信用评级因素 (0-10分)
        when(col("credit_mix") == "BAD", 10)
        .when(col("credit_mix") == "STANDARD", 5)
        .when(col("credit_mix") == "GOOD", 0)
        .otherwise(5) +
        
        # 信用历史因素 (0-5分) - 短历史加分，长历史减分
        when(col("credit_history_total_months") < 120, 5)  # 10年以下
        .when(col("credit_history_total_months") < 240, 3)  # 20年以下
        .otherwise(0) +
        
        # 支付行为因素 (0-5分)
        when(col("payment_behavior_spent_level") == "HIGH", 3).otherwise(0) +
        when(col("payment_behavior_value_level") == "LARGE", 2).otherwise(0) +
        
        # 收入因素 (0-5分) - 低收入加分
        when(col("annual_income") < 50000, 5)
        .when(col("annual_income") < 100000, 3)
        .otherwise(0) +
        
        # 贷款数量因素 (0-5分)
        when(col("total_loans") > 3, 5)
        .when(col("total_loans") > 1, 3)
        .otherwise(0) +
        
        # 逾期金额因素 (0-5分)
        when(col("max_overdue_amt") > 5000, 5)
        .when(col("max_overdue_amt") > 1000, 3)
        .when(col("max_overdue_amt") > 0, 1)
        .otherwise(0)
        
    ).cast("int")  # 转换为整数
)

# 确保评分在0-120范围内
gold_features_df = gold_features_df.withColumn(
    "default_risk_score",
    when(col("default_risk_score") > 120, 120)  # 如果超过120，设为120
    .when(col("default_risk_score") < 0, 0)     # 如果小于0，设为0
    .otherwise(col("default_risk_score"))
)

# 添加最终处理时间戳
gold_features_df = gold_features_df.withColumn(
    "gold_process_timestamp", 
    current_timestamp()
)

# 查看最终的Gold层特征表
print(f"\n最终Gold层特征表 - 行数: {gold_features_df.count()}, 列数: {len(gold_features_df.columns)}")

# 显示特征列和标签列
print("\nGold层特征和标签列示例:")
gold_features_df.select(
    "Customer_ID", 
    "has_clickstream_data",
    "age", 
    "annual_income", 
    "credit_history_total_months",
    "clickstream_features_imputed", 
    "num_defaulted_loans", 
    "max_missed_payments",
    "has_defaulted", 
    "default_risk_score"
).show(10)

# 分析标签分布
print("\n标签分布:")
gold_features_df.groupBy("has_defaulted").count().show()

# 风险评分分布
print("\n风险评分分布:")
gold_features_df.select("default_risk_score").summary("min", "25%", "50%", "75%", "max").show()

# 保存Gold层特征表
gold_features_df.write.mode("overwrite").parquet("datamart/gold/gold_features")
print("\nGold层特征表已保存到: datamart/gold/gold_features")

# 步骤5: 创建标签表 (用于机器学习模型训练)
print("\n步骤5: 创建标签表...")

# 从特征表中提取标签信息
gold_labels_df = gold_features_df.select(
    "Customer_ID",
    "has_defaulted",
    "default_risk_score",
    "has_clickstream_data",  # 用于区分完整数据和部分数据的客户
    "gold_process_timestamp"
)

# 保存Gold层标签表
gold_labels_df.write.mode("overwrite").parquet("datamart/gold/gold_labels")
print("\nGold层标签表已保存到: datamart/gold/gold_labels")

# 打印总结
print("\n=== Gold层处理完成 ===")
print(f"创建的特征表大小: {gold_features_df.count()} 行, {len(gold_features_df.columns)} 列")
print(f"创建的标签表大小: {gold_labels_df.count()} 行, {len(gold_labels_df.columns)} 列")
print(f"违约客户比例: {gold_labels_df.filter(col('has_defaulted') == True).count() / gold_labels_df.count() * 100:.2f}%")

加载Silver层表...
已加载 loan 表, 行数: 137500
已加载 attributes 表, 行数: 12500
已加载 financials 表, 行数: 12500
已加载 clickstream_detailed 表, 行数: 215376
已加载 clickstream_aggregated 表, 行数: 8974

步骤1: 创建基础客户表...
基础客户表中的客户数: 12500

点击流数据可用性分布:
+--------------------+-----+
|has_clickstream_data|count|
+--------------------+-----+
|                true| 8974|
|               false| 3526|
+--------------------+-----+


步骤2: 合并客户属性和财务数据...
客户属性和财务数据合并结果 - 行数: 12500, 列数: 18

步骤3: 添加点击流数据...
添加点击流数据后 - 行数: 12500, 列数: 35

步骤4: 添加贷款特征和创建标签...

最终Gold层特征表 - 行数: 12500, 列数: 45

Gold层特征和标签列示例:
+-----------+--------------------+---+------------------+---------------------------+----------------------------+-------------------+-------------------+-------------+------------------+
|Customer_ID|has_clickstream_data|age|     annual_income|credit_history_total_months|clickstream_features_imputed|num_defaulted_loans|max_missed_payments|has_defaulted|default_risk_score|
+-----------+--------------------+---+------------------+---